# 1 Creacion del Datamart

# 1.1 Set Up

In [30]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
#!pip install holidays
import holidays

%config IPComplete.greedy = True

pd.options.display.float_format = '{:15.2f}'.format

In [2]:
import sqlalchemy as sa
# Creacion de conexion
con = sa.create_engine('sqlite:///../Datos/ecommerce.db')

In [3]:
from sqlalchemy import inspect
# Creamos inspector
insp = inspect(con)
# Aplico inspector
insp.get_table_names()

['2019-Dec', '2019-Nov', '2019-Oct', '2020-Feb', '2020-Jan']

Tenemos 5 tablas con 5 meses.
Hay que importarlas y lo haremos por meses

# 1.2 Carga de Datos

In [4]:
oct = pd.read_sql('2019-Oct',con)
nov = pd.read_sql('2019-Nov',con)
dic = pd.read_sql('2019-Dec',con)
ene = pd.read_sql('2020-Jan',con)
feb = pd.read_sql('2020-Feb',con)

In [5]:
oct.head(1)

,index,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,68,2019-10-01 00:01:46 UTC,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361


In [6]:
feb.head(1)

,index,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,34,2020-02-01 00:01:43 UTC,view,5649305,1487580010335174721,None,concept,6.03,601680868,e896bd6d-804f-4664-9825-7a577116e427


Los archivos parecen tener la misma estructura.

Podemos integrarlos todos en un unico archivo antes de comenzar a hacer la calidad de datos.

**La integracion de los datos podemos hacerla apilando los registros ya que podemos ordenarlos por el campo event_time.**

# 1.3 Integracion de los datos

In [7]:
df = pd.concat([oct, nov, dic, ene, feb], axis=0)
df

,index,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,68,2019-10-01 00:01:46 UTC,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,72,2019-10-01 00:01:55 UTC,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,95,2019-10-01 00:02:50 UTC,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,122,2019-10-01 00:03:41 UTC,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,124,2019-10-01 00:03:44 UTC,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...,...,...,...
429785,4156660,2020-02-29 23:58:49 UTC,cart,5815662,1487580006317032337,None,None,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,4156663,2020-02-29 23:58:57 UTC,view,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,4156668,2020-02-29 23:59:05 UTC,cart,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,4156675,2020-02-29 23:59:28 UTC,view,5817692,1487580010872045658,None,None,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


# 1.4 Calidad de los datos

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2095076 entries, 0 to 429789
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   index          int64  
 1   event_time     object 
 2   event_type     object 
 3   product_id     int64  
 4   category_id    int64  
 5   category_code  object 
 6   brand          object 
 7   price          float64
 8   user_id        int64  
 9   user_session   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 175.8+ MB


- Eliminamos index.

- Convertimos event_time a datetime.

In [9]:
df.drop(columns = 'index', inplace = True)

- En dataset voluminosos como es el caso es mas agil dividir la cadena que forma la fecha.

- Unirlas de nuevo y aplicar el datetime con una funcion.

In [10]:
# Creamos funcion

def datetime_rapido(dt, formato):
    """
    Convierte una serie de strings que representan fechas y horas en objetos datetime de pandas,
    utilizando un formato especificado.

    Parámetros:
    -----------
    dt : pandas.Series
        Serie de strings con fechas y horas, cada uno en el formato 'YYYY-MM-DD HH:MM:SS' (u otro compatible con el formato dado).
    
    formato : str
        Cadena de formato compatible con datetime (por ejemplo, '%Y-%m-%d %H:%M:%S').

    Retorna:
    --------
    pandas.Series
        Serie de objetos datetime procesados con el formato dado.

    Notas:
    ------
    - La función interna `divide_fecha` divide cada string en fecha y hora, las une de nuevo 
      (útil en caso de strings con espacios extra u otras inconsistencias menores).
    - Se asume que cada string contiene al menos una fecha y una hora separadas por un espacio.
    """

    def divide_fecha(fecha):
        division = fecha.split()
        date = division[0]
        time = division[1]
        cadena = date +' '+ time
        return(cadena)

    resultado = pd.to_datetime(dt.apply(lambda x:divide_fecha(x)), format = formato)
    return(resultado)


In [11]:
formato = '%Y-%m-%d %H:%M:%S'

df.event_time = datetime_rapido(df.event_time, formato)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2095076 entries, 0 to 429789
Data columns (total 9 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[ns]
 1   event_type     object        
 2   product_id     int64         
 3   category_id    int64         
 4   category_code  object        
 5   brand          object        
 6   price          float64       
 7   user_id        int64         
 8   user_session   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 159.8+ MB


In [13]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5


## 1.4.1 Renombrado de variables

In [14]:
df.columns = ['fecha',
              'evento',
              'producto',
              'categoria',
              'categoria_cod',
              'marca',
              'precio',
              'usuario',
              'sesion']
df

,fecha,evento,producto,categoria,categoria_cod,marca,precio,usuario,sesion
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...,...,...
429785,2020-02-29 23:58:49,cart,5815662,1487580006317032337,None,None,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,2020-02-29 23:58:57,view,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,2020-02-29 23:59:05,cart,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,2020-02-29 23:59:28,view,5817692,1487580010872045658,None,None,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


## 1.4.2 Analisis de nulos

In [15]:
df.isna().sum().sort_values(ascending = False)

categoria_cod    2060411
marca             891646
sesion               506
fecha                  0
evento                 0
producto               0
categoria              0
precio                 0
usuario                0
dtype: int64

Tareas:

- categoria_cod y marca estan muy desinformadas y como no hay forma de reconstruirlas ni son relevantes se borran.

- sesion si es un campo relevante. Como son pocos valores 500 sobre 2 millones pueden eliminarse esos registros completos.

In [16]:
df.drop(columns =['categoria_cod','marca']).dropna()

,fecha,evento,producto,categoria,precio,usuario,sesion
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...
429785,2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,2020-02-29 23:59:28,view,5817692,1487580010872045658,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


## 1.4.3 Analisis de variables numericas

In [17]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
fecha,2095076,2019-12-16 20:38:28.450288384,2019-10-01 00:01:46,2019-11-08 18:37:10.750000128,2019-12-14 07:19:54,2020-01-26 06:52:57,2020-02-29 23:59:54,NaN
producto,2095076.00,5487129.98,3752.00,5724652.00,5811665.00,5858353.00,5932595.00,1300868.89
categoria,2095076.00,1553105502300814592.00,1487580004807082752.00,1487580005754995456.00,1487580008246412288.00,1487580013489291520.00,2242903426784559104.00,167899911509688000.00
precio,2095076.00,8.42,-47.62,2.05,4.00,6.86,327.78,19.14
usuario,2095076.00,521087186.59,4661182.00,480635722.00,553346925.00,578408245.00,622087993.00,87553241.57


Hay productos con precio negativo

In [18]:
df[df.precio <=0].producto.value_counts()

producto
5896186    79
5903915    50
5873428    37
5851294    29
5851304    29
           ..
5859332     1
5859329     1
5858607     1
5848998     1
5832437     1
Name: count, Length: 7049, dtype: int64

No se concentran en un unico producto o grupo de productos. Son pocos para el total y pueden eliminarse.

In [19]:
df = df[df.precio >0]
df.head(5)

,fecha,evento,producto,categoria,categoria_cod,marca,precio,usuario,sesion
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5


## 1.4.4 Analisis de variables categoricas

In [20]:
print('Cantidad de eventos:{}'.format(df.evento.nunique()))

Cantidad de eventos:4


In [21]:
print("Categoria de eventos: {}".format(list(df.evento.unique())))

Categoria de eventos: ['view', 'cart', 'remove_from_cart', 'purchase']


In [22]:
print('Cantidad de productos en catalogo: {}'.format(df.producto.nunique()))

Cantidad de productos en catalogo: 45329


In [23]:
print('Categorias de producto en catalogo: {}'.format(df.categoria.nunique()))

Categorias de producto en catalogo: 508


In [24]:
df.set_index('fecha', inplace = True)
df

,evento,producto,categoria,categoria_cod,marca,precio,usuario,sesion
fecha,,,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
2019-10-01 00:01:55,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2019-10-01 00:02:50,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
2019-10-01 00:03:41,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
2019-10-01 00:03:44,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...,...
2020-02-29 23:58:49,cart,5815662,1487580006317032337,None,None,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2020-02-29 23:58:57,view,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2020-02-29 23:59:05,cart,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90


# 1.5 Creacion de Variables

Tareas:

- Se va a desagregar las variables de tiempo.

- Incorporar el calendario de festivos local, en este caso el de Rusia, que es donde se radica el E-commerce

- Variables que recogen eventos exogenos. En este caso son promociones externas y como afectan a nuestras ventas.

## 1.5.1. Variables de tiempo

In [25]:
def componentes_fecha(dataframe):
    date = dataframe.index.date
    año = dataframe.index.year
    mes = dataframe.index.month
    dia = dataframe.index.day
    hora = dataframe.index.hour
    minuto = dataframe.index.minute
    segundo = dataframe.index.second
    
    
    return(pd.DataFrame({'date':date, 'año':año,'mes':mes, 'dia':dia, 'hora':hora, 'minuto':minuto, 'segundo':segundo}))

In [26]:
df = pd.concat([df.reset_index(),componentes_fecha(df)], axis = 1).set_index('fecha')
df.head()

,evento,producto,categoria,categoria_cod,marca,precio,usuario,sesion,date,año,mes,dia,hora,minuto,segundo
fecha,,,,,,,,,,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019,10,1,0,1,46
2019-10-01 00:01:55,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019,10,1,0,1,55
2019-10-01 00:02:50,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,2019-10-01,2019,10,1,0,2,50
2019-10-01 00:03:41,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df,2019-10-01,2019,10,1,0,3,41
2019-10-01 00:03:44,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,2019-10-01,2019,10,1,0,3,44


Añadimos festivos Rusia

In [46]:
festivos_RU = holidays.RU(years = 2020)

In [47]:
for fecha, fiesta in festivos_RU.items():
    print(fecha, fiesta)

2020-01-01 Новогодние каникулы
2020-01-02 Новогодние каникулы
2020-01-03 Новогодние каникулы
2020-01-04 Новогодние каникулы
2020-01-05 Новогодние каникулы
2020-01-06 Новогодние каникулы
2020-01-08 Новогодние каникулы
2020-01-07 Рождество Христово
2020-02-23 День защитника Отечества
2020-03-08 Международный женский день
2020-05-01 Праздник Весны и Труда
2020-05-09 День Победы
2020-06-12 День России
2020-11-04 День народного единства
2020-05-04 Выходной (перенесено с 04.01.2020)
2020-05-05 Выходной (перенесено с 05.01.2020)
2020-02-24 Выходной (перенесено с 23.02.2020)
2020-03-09 Выходной (перенесено с 08.03.2020)
2020-05-11 Выходной (перенесено с 09.05.2020)


In [48]:
df['festivo'] = df.date.apply(lambda x: 1 if (x in festivos_RU) else 0)

In [49]:
df

,evento,producto,categoria,categoria_cod,marca,precio,usuario,sesion,date,año,mes,dia,hora,minuto,segundo,festivo
fecha,,,,,,,,,,,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019,10,1,0,1,46,0
2019-10-01 00:01:55,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019,10,1,0,1,55,0
2019-10-01 00:02:50,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,2019-10-01,2019,10,1,0,2,50,0
2019-10-01 00:03:41,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df,2019-10-01,2019,10,1,0,3,41,0
2019-10-01 00:03:44,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,2019-10-01,2019,10,1,0,3,44,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 23:58:49,cart,5815662,1487580006317032337,None,None,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,49,0
2020-02-29 23:58:57,view,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,57,0
2020-02-29 23:59:05,cart,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,59,5,0


Comprobacion

In [50]:
df[df['festivo']==1].value_counts('date').sort_index()

date
2019-11-04    16432
2020-01-01     7644
2020-01-02    10776
2020-01-03    10618
2020-01-04    13086
2020-01-05    14558
2020-01-06    10622
2020-01-07    12922
2020-01-08    14007
2020-02-23     9818
2020-02-24    12726
Name: count, dtype: int64

Añadimos indicadores exogenos

In [53]:
df['black_friday'] = 0

df.loc['2019-11-29','black_friday'] = 1

df['san_valentin'] = 0

df.loc['2020-02-14', 'san_valentin'] = 1

In [54]:
df

,evento,producto,categoria,categoria_cod,marca,precio,usuario,sesion,date,año,mes,dia,hora,minuto,segundo,festivo,black_friday,san_valentin
fecha,,,,,,,,,,,,,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019,10,1,0,1,46,0,0,0
2019-10-01 00:01:55,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019,10,1,0,1,55,0,0,0
2019-10-01 00:02:50,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,2019-10-01,2019,10,1,0,2,50,0,0,0
2019-10-01 00:03:41,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df,2019-10-01,2019,10,1,0,3,41,0,0,0
2019-10-01 00:03:44,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,2019-10-01,2019,10,1,0,3,44,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 23:58:49,cart,5815662,1487580006317032337,None,None,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,49,0,0,0
2020-02-29 23:58:57,view,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,57,0,0,0
2020-02-29 23:59:05,cart,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,59,5,0,0,0


Comprobacion

In [55]:
df['black_friday'].value_counts()

black_friday
0    2052199
1      22333
Name: count, dtype: int64

In [56]:
df['san_valentin'].value_counts()

san_valentin
0    2062286
1      12246
Name: count, dtype: int64

In [58]:
variables = df.columns.to_list()
variables

['evento',
 'producto',
 'categoria',
 'categoria_cod',
 'marca',
 'precio',
 'usuario',
 'sesion',
 'date',
 'año',
 'mes',
 'dia',
 'hora',
 'minuto',
 'segundo',
 'festivo',
 'black_friday',
 'san_valentin']

In [59]:
orden = ['usuario',
         'sesion',
         'categoria',
         'evento',
         'producto',
         'precio']

orden

['usuario', 'sesion', 'categoria', 'evento', 'producto', 'precio']

In [60]:
resto =[nombre for nombre in variables if nombre not in orden]
resto

['categoria_cod',
 'marca',
 'date',
 'año',
 'mes',
 'dia',
 'hora',
 'minuto',
 'segundo',
 'festivo',
 'black_friday',
 'san_valentin']

In [61]:
df = df[orden + resto]
df

,usuario,sesion,categoria,evento,producto,precio,categoria_cod,marca,date,año,mes,dia,hora,minuto,segundo,festivo,black_friday,san_valentin
fecha,,,,,,,,,,,,,,,,,,
2019-10-01 00:01:46,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,1487580005092295511,view,5843665,9.44,None,f.o.x,2019-10-01,2019,10,1,0,1,46,0,0,0
2019-10-01 00:01:55,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,1487580013069861041,cart,5868461,3.57,None,italwax,2019-10-01,2019,10,1,0,1,55,0,0,0
2019-10-01 00:02:50,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,1487580006300255120,view,5877456,122.22,None,jessnail,2019-10-01,2019,10,1,0,2,50,0,0,0
2019-10-01 00:03:41,555448072,b5f72ceb-0730-44de-a932-d16db62390df,1487580013749338323,view,5649270,6.19,None,concept,2019-10-01,2019,10,1,0,3,41,0,0,0
2019-10-01 00:03:44,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,1487580005411062629,view,18082,16.03,None,cnd,2019-10-01,2019,10,1,0,3,44,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 23:58:49,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,1487580006317032337,cart,5815662,0.92,None,None,2020-02-29,2020,2,29,23,58,49,0,0,0
2020-02-29 23:58:57,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,1487580006317032337,view,5815665,0.59,None,None,2020-02-29,2020,2,29,23,58,57,0,0,0
2020-02-29 23:59:05,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,1487580006317032337,cart,5815665,0.59,None,None,2020-02-29,2020,2,29,23,59,5,0,0,0


In [62]:
df.to_pickle('../Datos/tablon_analitico.pickle')